# 连接

+ 初次使用通过 usb 连接电脑，通过 Tera Term 软件连接，将 XIA NET 内置 ubuntu 系统的 ip 地址固定（192.168.1.86）。随后将 XIA NET 和获取电脑连接到同一交换机上，设置获取电脑相应的 ip 地址（192.168.1.100）。
+ 通过网页/终端 ssh 均可以进入 XIA NET 获取。网页端进入在浏览器输入 192.168.1.86，用户名 root，密码 xia17pxn 即可登录；终端 ssh 输入 ssh -Y root@192.168.1.86，密码 xia17pxn。

# 注意
+ 网页端的的工作目录在 /var/www/webops 中，shell 端的工作路径为 /var/www 中，为了方便，将 网页端目录下的 setting.ini 文件做软链接至 shell 端工作路径下，但两者生成的数据文件仍在各自文件夹中。
+ 建议在 /var/www/webops/ 下修改 setting.ini 文件，网页端获取数据，网页端下载数据，但需要注意如果下载了 MCA 谱，需及时删除，否则在线谱会优先读取默认路径下的 MCA 谱，也即在实际获取时无法正确显示在线谱。
+ 符合模式下，4 个通道的 good channel 均需要打开，否则无数据记录。
+ 0x402 模式运行时，网页的在线能谱显示有问题，应该是网页的 bug，但是在 0x503 模式下运行正常。

# 参数设置 setting.ini

## 具体需参考手册，以下仅对部分常用参数作说明。
| 参数名称     | 描述 |
| :-----------  | :----------- |
| REQ_RUNTIME    | 获取时间（s）  |
| MCSRB_TERM01_01 | 一般取 1，表示 ch0 和 ch1 为 50 $\Omega$ 阻抗匹配 |
| MCSRB_TERM23_02 | 一般取 1，表示 ch2 和 ch3 为 50 $\Omega$ 阻抗匹配 |
| COINC_PATTERN_xxxx  | 举例：<br> COINC_PATTERN_0011=1 表示 ch0 和 ch1 符合测量；<br> COINC_PATTERN_0111=1 & <br> COINC_PATTERN_1011=1 & <br> COINC_PATTERN_1101=1 & <br> COINC_PATTERN_1110=1 表示 ch0-3 有三个点火时记录，但是不区分是哪三个通道点火。|  
| COINCIDENCE_WINDOW | 符合时间窗（$\mu$s）
| RUN_TYPE   | 获取类型        |
| CCSRA_GROUP_00| 触发类型，self trigger 时取 0；group trigger 时取 1 |
| CCSRA_GOOD_02 | good channel，取 0 则不处理该通道 |
| CCSRA_TRIGENA_04 | 取 0 表示使用外部触发；取 1，表示允许 self 和 group trigger |
| CCSRA_INVERT_05 | 取 1 表示反相信号 |
| ANALOG_GAIN | 增益相关，可取 2/5 |
| DIG_GAIN | 增益相关，可取正浮点数 |
| VOFFSET | 基线 offset |
| TRACE_LENGTH | 0$\sim$16 $\mu$s，4 ns/point |
| TRACE_DELAY | 0$\sim$16 $\mu$s，4 ns/point |

## 不同类型探测器参数举例
### LaBr$_3$ 探测器
+ 能量参数

| 参数名称     | 取值 |
| :-----------  | :----------- |
| FILTER_RANGE | 1 |
| ENERGY_RISETIME | 0.144 $\mu$s | 
| ENERGY_FLATTOP | 0.048 $\mu$s |
| TAU | 0.024 $\mu$s |
+ 触发参数

| 参数名称     | 取值 |
| :-----------  | :----------- |
| TRIGGER_RISETIME | 0.008 $\mu$s |
| TRIGGER_FLATTOP | 0.016 $\mu$s |
| TRIGGER_THRESHOLD | 1 |

## RUN_TYPE
| run type | file format | waveform | daq function | file extension | note |
| :-----------  | :----------- | :-----------  | :----------- | :-----------  | :----------- |
| 0x400 | binary | $\checkmark$ |startdaq / acquire | .b00 | list mode |
| 0x402 | binary | $\checkmark$ |acquire | .b00 | coincidence list mode |
| 0x500 | text | $\checkmark$ | startdaq | .txt | list mode |
| 0x501 | text | $\times$ | startdaq | .dat | list mode |
| 0x502 | text | $\times$ | startdaq | .dt2 | list mode with pluse shape analysis |
| 0x503 | text | $\times$ | coincdaq | .dt3 | coincidence list mode |

# 数据处理
## run type 501
+ 保存的数据为 .dat 后缀，运行 dat2tree 程序后可得到对应的 root 文件，保存的是 tree 结构。具体运行方式如下：
```
./dat2tree 1 //处理 run0001.dat 文件
```
```
root -l rootfile/run0001_0x501.root  // 进入 root 
tr->Draw("adc>>(4096,0,32768)")    // 绘制能谱
```

## run_type 401
+ 保存的数据为 .b00 后缀，运行 bin2tree 程序后可得到对应的 root 文件，保存的是 tree 结构。具体运行方式如下：
```
./bin2tree 1 //处理 run0001.b00 文件
```
```
root -l rootfile/run0001_0x400.root  // 进入 root 
tr->Draw("v_trace:Iteration$","","",10,0)    // 绘制波形
tr->Draw("Sum$(v_trace)-Length$(v_trace)*(v_trace[0]+v_trace[1]+v_trace[2]+v_trace[3]+v_trace[4]+v_trace[5]+v_trace[6]+v_trace[7]+v_trace[8]+v_trace[9]+v_trace[10]+v_trace[11]+v_trace[12]+v_trace[13]+v_trace[14]+v_trace[15]+v_trace[16]+v_trace[17]+v_trace[18]+v_trace[19])/20.>>(4096,0,32768)") // 绘制能谱
```